# 최종 전처리 코드
1. title에 [북한 날씨] 라는 단어가 들어가는 기사를 삭제하기. 
2. 똑같은 title과 본문을 가진 뉴스기사를 삭제하기

In [2]:
import json
import re
import unicodedata

# ----------------------------
# 1. JSON 파일 불러오기
# ----------------------------
INPUT_PATH = "/home/ds4_sia_nolb/#FINAL_POLARIS/04_plus_preprocessing/preprocessing_filter_data/ten_year_dprk.json"
OUTPUT_PATH = "/home/ds4_sia_nolb/#FINAL_POLARIS/04_plus_preprocessing/preprocessing_final_data/re_final_preprocessing.json"

# ----------------------------
# 2. 함수 정의
# ----------------------------
ZERO_WIDTH_PATTERN = re.compile(r"[\u200B-\u200D\uFEFF]")

# [북한날씨], [북한 날씨], <북한날씨>, <북한 날씨> 모두 매칭
WEATHER_REGEX = re.compile(r"[\[<]북한\s*날씨[\]>]")

def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data if isinstance(data, list) else [data]

def extract_title(doc):
    return (doc.get("metadata", {}).get("title")
            or doc.get("title")
            or "")

def extract_text(doc):
    txt = doc.get("text")
    if isinstance(txt, str) and txt.strip():
        return txt
    txt2 = doc.get("text_resource", {}).get("text")
    return txt2 if isinstance(txt2, str) else ""

def normalize(s: str) -> str:
    s = unicodedata.normalize("NFKC", s)
    s = ZERO_WIDTH_PATTERN.sub("", s)
    return s.strip()

def is_weather_tagged(title: str) -> bool:
    t = normalize(title)
    return bool(WEATHER_REGEX.search(t))

def normalize_for_dup(s: str) -> str:
    s = unicodedata.normalize("NFKC", s)
    s = ZERO_WIDTH_PATTERN.sub("", s)
    s = s.lower()
    s = re.sub(r"\s+", " ", s)
    return s.strip()

# ----------------------------
# 3. 데이터 로드
# ----------------------------
docs = load_json(INPUT_PATH)
print(f"원본 문서 개수: {len(docs)}")


# ----------------------------
# 4. [북한 날씨] 제목 필터링
# ----------------------------
removed_weather = 0
examples = []
kept_after_weather = []

for d in docs:
    title = extract_title(d)
    if is_weather_tagged(title):
        removed_weather += 1
        if len(examples) < 10:
            examples.append(normalize(title))
        continue
    kept_after_weather.append(d)

print(f"[북한날씨]/[북한 날씨] 제목 제거 수: {removed_weather}")
if examples:
    print("제거된 제목 예시(최대 10개):")
    for t in examples:
        print(" -", t)
print(f"제거 후 문서 개수: {len(kept_after_weather)}")



# ----------------------------
# 5. (title + text) 중복 제거
# ----------------------------
seen = set()
unique_docs = []
dup_removed = 0

for d in kept_after_weather:
    title = extract_title(d)
    text = extract_text(d)
    key = (normalize_for_dup(title), normalize_for_dup(text))
    if key in seen:
        dup_removed += 1
        continue
    seen.add(key)
    unique_docs.append(d)

print(f"중복 제거: {dup_removed}건")
print(f"최종 문서 개수: {len(unique_docs)}")


# ----------------------------
# 6. 저장
# ----------------------------
with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(unique_docs, f, ensure_ascii=False, indent=2)

print(f"전처리 완료 및 저장 경로: {OUTPUT_PATH}")


원본 문서 개수: 84230
[북한날씨]/[북한 날씨] 제목 제거 수: 3443
제거된 제목 예시(최대 10개):
 - [북한날씨] 대체로 맑음...양강도 약간의 눈
 - [북한날씨] 대체로 맑음...양강도 일부 한때 눈
 - [북한날씨] 대체로 맑음...양강도 일부 흐림
 - [북한날씨] 대체로 맑아
 - [북한날씨] 대체로 맑음...함북 일부 한때 눈
 - [북한날씨] 대체로 맑음
 - [북한날씨] 대체로 맑다가 오후부터 흐려져
 - [북한날씨] 밤에 대부분 지역 눈
 - [북한날씨] 구름 많음...동해안 대체로 맑음
 - [북한날씨] 가끔 구름...낮 최고기온 영하권
제거 후 문서 개수: 80787
중복 제거: 353건
최종 문서 개수: 80434
전처리 완료 및 저장 경로: /home/ds4_sia_nolb/#FINAL_POLARIS/04_plus_preprocessing/preprocessing_final_data/re_final_preprocessing.json
